# ДЗ к уроку 3
Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

# Импорты

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import re
from html.parser import HTMLParser
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from string import punctuation
from pymystem3 import Mystem
from gensim.models import Word2Vec, FastText
import annoy
import tqdm
from pymorphy2 import MorphAnalyzer

# Исходные данные

In [32]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative])
df = df.reset_index().drop('index', axis=1)

# Preprocessing

In [33]:
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def preprocessing(corpus):
    
    corpus = corpus.apply(str.lower)
    
    corpus = corpus.apply(lambda x: re.sub(r'@[\w]*', '', x))
#     Заменим пунктуацию на пробелы
    corpus = corpus.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
#     Заменим спец. символы на пробелы
    corpus = corpus.apply(lambda x: re.sub(r'[^a-zA-Zа-яА-Я0-9ё]', ' ', x))
#     Заменим числа на пробелы
    corpus = corpus.apply(lambda x: re.sub(r'[^a-zA-Zа-яА-Яё]', ' ', x))
#     Уберем слова менее 3 символов
    corpus = corpus.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
#     Уберем стоп-слова
    corpus = corpus.apply(lambda x: ' '.join([w for w in x.split() if not w in stop_words]))
                          
    return corpus

In [35]:
t = preprocessing(df['text'])[:1000]
t

0      школота поверь самое общество профилирующий пр...
1                       таки немного похож мальчик равно
2                                     идиотка испугалась
3      углу сидит погибает голода ещё порции взяли хо...
4      значит страшилка блин посмотрев части создастс...
                             ...                        
995                        рискни рамку готовлю блинчики
996          спасибо считаю найти свое место очень важно
997    уаааа около прошел паренек чисто женской поход...
998           монгол коментийн vзэж жаахан хэрэгт дурлая
999              анютка любит поезда осторожнее ирисками
Name: text, Length: 1000, dtype: object

In [36]:
def preprocess_txt(line):
    morpher = MorphAnalyzer()
    
    spls = "".join(i for i in line.strip()).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stop_words and i != ""]
    return spls

# Модель

In [37]:
modelW2V = Word2Vec(sentences=t, vector_size=300, window=5, min_count=1)

In [38]:
modelFT = FastText(sentences=t, vector_size=300, min_count=1, window=5, workers=8)

In [39]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in tqdm.tqdm_notebook(t):
    try:
        n_w2v = 0
        n_ft = 0
        spls = line.split()
    #     index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])

        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1

        if counter > 100000:
            break
    except:
        print(f'Error! Count: {counter} line: "{line}"')

w2v_index.build(10)
ft_index.build(10)

C:\Users\Admin\AppData\Local\Temp\ipykernel_20816\3581189489.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm.tqdm_notebook(t):


  0%|          | 0/1000 [00:00<?, ?it/s]

True

In [40]:
def get_response(question, index, model):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    return index.get_nns_by_vector(vector, 5 )
    

# Результат

Получим твиты ближайшие к запросу

In [42]:
# Запрос
df.loc[0, 'text']

'@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'

In [43]:
df.loc[get_response(t[0], w2v_index, modelW2V), 'text']

1     Да, все-таки он немного похож на него. Но мой ...
5     ну любишь или нет? — Я не знаю кто ты бля:D ht...
9     Теперь у меня есть частичка Сиднея :) #Sydney ...
11    RT @dicyziqecida: Как-то я забыла, что вчера п...
18    @xLesherx @4EU3 зря вы с этой хуйней шутите)) ...
Name: text, dtype: object